<a href="https://colab.research.google.com/github/CodeWithSridhar/Innomatics-Internship/blob/main/Innomatics_Uber_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = 'https://raw.githubusercontent.com/CodeWithSridhar/Innomatics-Internship/main/uber_rides_data.xlsx%20-%20sample_train.csv'

In [6]:
df = pd.read_csv(data)

In [7]:
df.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


### 1. What is the shape of given dataset?`


In [8]:
df.shape

(200000, 8)

### 2. How many integer columns(by default) are given in the dataset?

In [9]:
# Select integer columns (by default)
int_columns = df.select_dtypes(include=['int64'])

# Get the number of integer columns
num_int_columns = int_columns.shape[1]

print("Number of integer columns in the dataset:", num_int_columns)


Number of integer columns in the dataset: 2


### 3. How many missing values exists in 'dropoff_longitude' column?

In [10]:
# Count missing values in the 'dropoff_longitude' column
missing_values = df['dropoff_longitude'].isna().sum()

print("Number of missing values in 'dropoff_longitude' column:", missing_values)


Number of missing values in 'dropoff_longitude' column: 1


### 4. What is the data type of ' pickup_datetime' feature in your data?

In [11]:
# Get the data type of the 'pickup_datetime' feature
pickup_datetime_dtype = df['pickup_datetime'].dtype

print("Data type of 'pickup_datetime' feature:", pickup_datetime_dtype)


Data type of 'pickup_datetime' feature: object


### 5.correct syntax to convert 'pickup_datetime' to datetime datatype?

In [12]:
# Convert 'pickup_datetime' to datetime data type
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Now 'pickup_datetime' is of datetime data type
print(df['pickup_datetime'].dtype)

datetime64[ns, UTC]


### What is the average fare amount?
*
Remove the null values from the dataframe to answer the following question.

In [13]:
# Remove rows with null values
df_cleaned = df.dropna(subset=['fare_amount'])

# Calculate the average fare amount
average_fare = df_cleaned['fare_amount'].mean()

print("Average fare amount:", average_fare)


Average fare amount: 11.359955250000002


### Calculate distance between each pickup and dropoff points using Haversine formula.
What is the median haversine distance between pickup and dropoff location according to the given dataset?
*
Read about Haversine Distance here: https://en.wikipedia.org/wiki/Haversine_formula

In [14]:
# Function to calculate Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    r = 6371  # Radius of the Earth in kilometers
    distance = r * c

    return distance

# Calculate Haversine distance for each row
df['haversine_distance'] = df.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'],
                                                          row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

# Calculate the median Haversine distance
median_haversine_distance = df['haversine_distance'].median()

print("Median Haversine distance between pickup and dropoff locations:", median_haversine_distance, "kilometers")


Median Haversine distance between pickup and dropoff locations: 2.1209923961833708 kilometers


### What is the maximum haversine distance between pickup and dropoff location according to the given dataset?

In [15]:
# Find the maximum Haversine distance
max_haversine_distance = df['haversine_distance'].max()

print("Maximum Haversine distance between pickup and dropoff locations:", max_haversine_distance, "kilometers")


Maximum Haversine distance between pickup and dropoff locations: 16409.239135313168 kilometers


### How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset?

In [16]:
# Count the number of rides with 0.0 Haversine distance
zero_distance_rides = df[df['haversine_distance'] == 0.0]
num_zero_distance_rides = zero_distance_rides.shape[0]

print("Number of rides with 0.0 Haversine distance between pickup and dropoff locations:", num_zero_distance_rides)


Number of rides with 0.0 Haversine distance between pickup and dropoff locations: 5632


### What is the mean 'fare_amount' for rides with 0 haversine distance?

Do you sense something fishy? Try to analyze, and give your expert opinion in Notebook.

In [17]:
# Calculate the mean 'fare_amount' for rides with 0.0 Haversine distance
mean_fare_for_zero_distance_rides = zero_distance_rides['fare_amount'].mean()

print("Mean 'fare_amount' for rides with 0.0 Haversine distance:", mean_fare_for_zero_distance_rides)


Mean 'fare_amount' for rides with 0.0 Haversine distance: 11.585317826704546


    Having a 0.0 Haversine distance for rides suggests that the pickup and dropoff locations are very close or even identical. This is unusual because it implies that passengers are being charged a fare amount for a ride that did not actually occur or that the data might contain some errors.

    It's important to investigate these cases further to determine the cause of these 0.0 distance rides. Some possible explanations could include:

      1. Data entry errors, such as incorrect latitude and longitude coordinates for pickup and dropoff locations.
      2. Rides that were canceled immediately after pickup.
      3. Rides where the pickup and dropoff locations are very close, such as within the same building or street, but still incur a minimum fare.

### What is the maximum 'fare_amount' for a ride?


In [18]:
max_fare_amount = df['fare_amount'].max()

print("Maximum 'fare_amount' for a ride:", max_fare_amount)


Maximum 'fare_amount' for a ride: 499.0


### What is the haversine distance between pickup and dropoff location for the costliest ride?
*
Do you sense something fishy? Try to analyze, and give your expert opinion

In [19]:
# Find the row with the highest 'fare_amount'
costliest_ride_row = df[df['fare_amount'] == df['fare_amount'].max()]

# Calculate the Haversine distance for the costliest ride
haversine_distance_costliest_ride = haversine(costliest_ride_row['pickup_latitude'].values[0],
                                              costliest_ride_row['pickup_longitude'].values[0],
                                              costliest_ride_row['dropoff_latitude'].values[0],
                                              costliest_ride_row['dropoff_longitude'].values[0])

print("Haversine distance for the costliest ride:", haversine_distance_costliest_ride, "kilometers")


Haversine distance for the costliest ride: 0.0007899213191009993 kilometers


    Having an unusually high 'fare_amount' for a ride could indicate several potential issues or anomalies in the data:

      1. **Data Entry Errors**: There might be data entry errors leading to an artificially high fare amount.

      2. **Outliers**: The costliest ride might be a legitimate outlier, such as a long-distance trip or a ride with multiple stops.

      3. **Pricing Model Issues**: There could be issues with the pricing model, leading to overcharging.

      4. **Fraud or Anomalies**: It's also possible that this high fare is a result of fraudulent activity or anomalies.

### How many rides were recorded in the year 2014?

In [20]:
# Extract the year from 'pickup_datetime'
df['pickup_year'] = df['pickup_datetime'].dt.year

# Count the number of rides recorded in 2014
rides_in_2014 = df[df['pickup_year'] == 2014]
num_rides_in_2014 = rides_in_2014.shape[0]

print("Number of rides recorded in the year 2014:", num_rides_in_2014)


Number of rides recorded in the year 2014: 29968


### How many rides were recorded in the first quarter of 2014?

In [21]:
# Extract the quarter from 'pickup_datetime'
df['pickup_quarter'] = df['pickup_datetime'].dt.quarter

# Count the number of rides recorded in the first quarter of 2014 (quarter = 1)
rides_in_first_quarter_2014 = df[(df['pickup_year'] == 2014) & (df['pickup_quarter'] == 1)]
num_rides_in_first_quarter_2014 = rides_in_first_quarter_2014.shape[0]

print("Number of rides recorded in the first quarter of 2014:", num_rides_in_first_quarter_2014)


Number of rides recorded in the first quarter of 2014: 7687


### On which day of the week in September 2010, maximum rides were recorded ?

In [22]:
# Extract the year and month from 'pickup_datetime'
df['pickup_year'] = df['pickup_datetime'].dt.year
df['pickup_month'] = df['pickup_datetime'].dt.month

# Filter rides for September 2010 (year = 2010, month = 9)
rides_in_september_2010 = df[(df['pickup_year'] == 2010) & (df['pickup_month'] == 9)]

# Find the day of the week with the maximum number of rides
max_rides_day_of_week = rides_in_september_2010['pickup_datetime'].dt.day_name().mode().values[0]

print("Day of the week in September 2010 with the maximum rides:", max_rides_day_of_week)


Day of the week in September 2010 with the maximum rides: Thursday


### Apply a Machine Learning Algorithm to predict the fare amount given following input features:
passenger_count, distance and ride_week_day.

Perform a 70-30 split of data.

Which algorithm gives the least adjusted R square value?

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer


# Select features and target variable (excluding 'ride_week_day')
X = df[['passenger_count', 'haversine_distance']]
y = df['fare_amount']

# Impute missing values (NaN) with mean for numerical features
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and test sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize regression models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}

# Train and evaluate models
adjusted_r2_scores = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate the R-squared value
    r2 = r2_score(y_test, y_pred)

    # Calculate adjusted R-squared
    n = len(X_test)
    p = X_test.shape[1]
    adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

    adjusted_r2_scores[model_name] = adjusted_r2

# Find the algorithm with the least adjusted R-squared value
least_adjusted_r2_model = min(adjusted_r2_scores, key=adjusted_r2_scores.get)
least_adjusted_r2_value = adjusted_r2_scores[least_adjusted_r2_model]

print("Algorithm with the least adjusted R-squared value:", least_adjusted_r2_model)
print("Least adjusted R-squared value:", least_adjusted_r2_value)


Algorithm with the least adjusted R-squared value: Linear Regression
Least adjusted R-squared value: 0.0006828926442224992
